# <span style = "color : green"> <h2 style="text-align: center;"> Used Car price prediction Experiment </h2> </span>
 <h3 style="text-align: center;"> Let's GO </h3>

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

### As per we are using the clean data so need to preprocess 

the originar data : https://www.kaggle.com/datasets/rupeshraundal/marketcheck-automotive-data-us-canada?select=ca-dealers-used.csv

In [4]:
df=pd.read_csv('../NOTEBOOK/data/used_car_canada.csv')
df.sample(5)

,price,miles,year,make,model,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block,state
61185,44695.0,61000.0,2017.0,ford,F-350 Super Duty,pickup,Truck,4WD,Automatic,gasoline,6.2,V,AB
177417,24450.0,14966.0,2019.0,subaru,Impreza,sedan,Car,4WD,Automatic,gasoline,2.0,H,ON
89491,39785.0,22673.0,2019.0,jeep,Wrangler,suv,Truck,4WD,Manual,gasoline,3.6,V,QC
158069,17985.0,27040.0,2020.0,kia,Rio 5-Door,hatchback,Car,FWD,Automatic,gasoline,1.6,I,AB
76728,17795.0,93694.0,2017.0,mazda,CX-5,suv,Truck,4WD,Automatic,gasoline,2.5,I,QC


In [6]:
cols=df.columns
cols

Index(['price', 'miles', 'year', 'make', 'model', 'body_type', 'vehicle_type',
       'drivetrain', 'transmission', 'fuel_type', 'engine_size',
       'engine_block', 'state'],
      dtype='object')

In [9]:
cols_to_drop = ['body_type', 'vehicle_type', 'drivetrain', 'transmission', 'fuel_type', 'engine_block']
df = df.drop(cols_to_drop, axis=1)

In [10]:
df.head()

,price,miles,year,make,model,engine_size,state
0,179999.0,9966.0,2017.0,acura,NSX,3.5,NB
1,179995.0,5988.0,2017.0,acura,NSX,3.5,QC
2,168528.0,24242.0,2017.0,acura,NSX,3.5,BC
3,220000.0,6637.0,2020.0,acura,NSX,3.5,ON
4,220000.0,6637.0,2020.0,acura,NSX,3.5,ON


In [12]:
df_toyota_honda = df.loc[(df['make'] == 'honda') | (df['make'] == 'toyota')]

In [26]:
model_file_path=Path('../NOTEBOOK/data/honda_toyota_ca.csv')

if model_file_path.exists():
    print("model file exist , no need to recreate....")
else:
    df_toyota_honda.to_csv('../NOTEBOOK/data/honda_toyota_ca.csv', index=False, header=True)
    print("model file saved successfully......")

model file exist , no need to recreate....


In [28]:
df = pd.read_csv('../NOTEBOOK/data/honda_toyota_ca.csv')
df.sample(5)

,price,miles,year,make,model,engine_size,state
37102,6888.0,189260.0,2008.0,toyota,Tercel,1.5,AB
21406,12900.0,142000.0,2012.0,honda,CR-V,2.4,ON
11990,16995.0,36546.0,2018.0,toyota,Corolla,1.8,ON
30873,45990.0,49700.0,2019.0,toyota,Tacoma,3.5,NB
19598,27789.0,48141.0,2019.0,honda,CR-V,1.5,ON


In [15]:
from sklearn.model_selection import train_test_split

X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df[['make', 'model']], test_size=0.2, shuffle=True, random_state=42)

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor

cat_index = [2,3,5]

cat_features_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_features_transformer, cat_index)
    ]
)


model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", GradientBoostingRegressor(random_state=42))
    ]
)

In [17]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  [2, 3, 5])])),
                ('regressor', GradientBoostingRegressor(random_state=42))])

In [29]:
X_test

,miles,year,make,model,engine_size,state
33579,109114.0,2015.0,toyota,Camry,2.5,BC
33456,97881.0,2007.0,toyota,Camry,2.4,BC
17549,116798.0,2015.0,honda,CR-V,2.4,NS
16324,67316.0,2016.0,honda,CR-V,2.4,ON
36808,44916.0,2019.0,toyota,4Runner,4.0,ON
...,...,...,...,...,...,...
35884,23280.0,2020.0,toyota,Corolla Hatchback,2.0,BC
3327,83651.0,2016.0,honda,Civic,1.5,BC
2699,144250.0,2015.0,honda,Civic,1.8,ON
12761,34700.0,2018.0,toyota,Corolla,1.8,QC


In [18]:
model.score(X_test, y_test)

0.4865534075230413

In [19]:
from joblib import dump

In [25]:
model_file_path=Path("../NOTEBOOK/model/car_model.joblib")

if model_file_path.exists():
    print("model file exist , no need to recreate....")
else:
    dump(model,'../NOTEBOOK/model/car_model.joblib')
    print("model file saved successfully......")

model file exist , no need to recreate....
